In [1]:
import os
import torch
import numpy as np
import tifffile
import pandas as pd
import pkg_resources
from cellpose import models
from skimage.measure import regionprops, label
from tqdm import tqdm



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.8.20 
torch version:  	2.4.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [2]:
# Check CUDA and GPU availability
print(f"CUDA is available: {torch.cuda.is_available()}")
print(pkg_resources.get_distribution("cellpose").version)
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

CUDA is available: True
4.0.6
GPU Name: NVIDIA RTX 4500 Ada Generation


In [3]:
# PARAMETERS - update these paths
input_folder = r"L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked"
output_folder = r"L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked\segmentation\cellposeSAM_gpu_results"
os.makedirs(output_folder, exist_ok=True)

In [4]:
# The new Cellpose-SAM model is loaded via model_type = 'sam'
model = models.CellposeModel(gpu=torch.cuda.is_available(), pretrained_model='cpsam')

In [5]:
# Segmentation parameters
torch.set_num_threads(36)
os.environ["OMP_NUM_THREADS"] = "36"
os.environ["MKL_NUM_THREADS"] = "36"

print(f"PyTorch using {torch.get_num_threads()} threads")

PyTorch using 36 threads


In [6]:
diameter = None  # cell diameter; set to None for automatic
batch_size = 128
tile_overlap = 0.05
bsize = 256  # tile size for Cellpose

In [7]:
# Set PyTorch environment variable to reduce CUDA memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"# Utility function: shape features from regionprops
def shape_features(region):
    area = region.area
    perimeter = region.perimeter if region.perimeter > 0 else 1
    roundness = 4 * np.pi * area / (perimeter ** 2)
    major_axis = region.major_axis_length
    minor_axis = region.minor_axis_length
    aspect_ratio = major_axis / minor_axis if minor_axis != 0 else 0
    return {
        "Area": area,
        "Length": major_axis,
        "Width": minor_axis,
        "Roundness": roundness,
        "Aspect Ratio": aspect_ratio,
    }

In [8]:
def analyze_single_stack(stack_path):
    stack = tifffile.imread(stack_path)

    # Expect shape (2, H, W): DIC and fluorescence
    if stack.ndim == 3 and stack.shape[0] == 2:
        dic_img = stack[0]
        fluo_img = stack[1]
    else:
        raise ValueError(f"Unexpected image shape {stack.shape} in {stack_path}")

    # Normalize each image independently
    def normalize_channel(img):
        mn, mx = img.min(), img.max()
        return (img - mn) / (mx - mn) if mx > mn else img

    dic_img = normalize_channel(dic_img)
    #fluo_img = normalize_channel(fluo_img)

    # Prepare input for Cellpose-SAM: (channels, H, W)
    img_cp = np.expand_dims(dic_img, axis=0)

    # Pad to multiple of tile size
    def pad_to_multiple(img, multiple):
        c, h, w = img.shape
        pad_h = (multiple - h % multiple) % multiple
        pad_w = (multiple - w % multiple) % multiple
        img_padded = np.pad(img, ((0, 0), (0, pad_h), (0, pad_w)), mode='reflect')
        return img_padded, (h, w)  # original height and width

    img_cp, orig_shape = pad_to_multiple(img_cp, bsize)

    # Run Cellpose-SAM model
    masks_padded, flows, styles = model.eval(
        img_cp,
        diameter=diameter,
        batch_size=batch_size,
        augment=False,
        #Turn this to true if you want better quality segmentation
        tile_overlap=tile_overlap,
        bsize=bsize,
    )

    # Crop masks back to original shape, handle 2D or 3D masks output
    if masks_padded.ndim == 3:  # (batch, H, W)
        masks = masks_padded[0][:orig_shape[0], :orig_shape[1]]
    elif masks_padded.ndim == 2:  # (H, W) no batch dim
        masks = masks_padded[:orig_shape[0], :orig_shape[1]]
    else:
        raise ValueError(f"Unexpected masks_padded shape: {masks_padded.shape}")

    # Sanity check
    if masks.shape != dic_img.shape:
        raise ValueError(f"Mismatch after cropping: masks shape {masks.shape} vs dic_img {dic_img.shape}")

    # Label mask for regionprops analysis
    labeled_masks = label(masks)
    props = regionprops(labeled_masks, intensity_image=fluo_img)

    # Background intensity
    background_mask = masks == 0
    background_values = fluo_img[background_mask]
    background_mean = float(np.mean(background_values)) if background_values.size > 0 else 0
    background_std = float(np.std(background_values)) if background_values.size > 0 else 0

    # Extract features
    data = []
    for prop in props:
        feats = shape_features(prop)
        data.append({
            "Label": prop.label,
            "Area": feats["Area"],
            "Length": feats["Length"],
            "Width": feats["Width"],
            "Roundness": feats["Roundness"],
            "Aspect Ratio": feats["Aspect Ratio"],
            "Mean Intensity": prop.mean_intensity,
            "Centroid X": prop.centroid[1],
            "Centroid Y": prop.centroid[0],
            "Background Mean": background_mean,
            "Background Std": background_std,
        })

    df = pd.DataFrame(data)
    return df, masks.astype(np.uint16)

In [9]:
# GPU memory check and parameter adjustment for OOM errors
def get_free_gpu_memory():
    torch.cuda.empty_cache()
    return torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()

def adjust_parameters_on_oom(current_batch, current_bsize):
    free_mem = get_free_gpu_memory()
    print(f"Free GPU memory: {free_mem / (1024**3):.2f} GB")
    if free_mem < 5 * (1024**3):
        new_batch = max(64, current_batch // 2)
        new_bsize = max(128, current_bsize // 2)
        print(f"Reducing batch size from {current_batch} to {new_batch} and tile size from {current_bsize} to {new_bsize}")
        return new_batch, new_bsize
    return current_batch, current_bsize

In [10]:
def analyze_single_stack_with_retries(stack_path, retries=3):
    global batch_size, bsize
    for attempt in range(retries):
        try:
            return analyze_single_stack(stack_path)
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"⚠️ CUDA OOM on attempt {attempt + 1} for {os.path.basename(stack_path)}. Adjusting parameters...")
                batch_size, bsize = adjust_parameters_on_oom(batch_size, bsize)
                torch.cuda.empty_cache()
            else:
                raise
    raise RuntimeError(f"Failed to process {stack_path} after {retries} attempts due to OOM.")

In [11]:
# Main batch analysis loop with progress bar
tiff_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".tif", ".tiff"))]
print(f"Found {len(tiff_files)} TIFF stacks for analysis.")

with tqdm(total=len(tiff_files), desc="Processing TIFF stacks", dynamic_ncols=True) as pbar:
    for fname in tiff_files:
        fpath = os.path.join(input_folder, fname)
        try:
            df, masks = analyze_single_stack_with_retries(fpath)

            excel_path = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_results.xlsx")
            df.to_excel(excel_path, index=False)

            masks_path = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_masks.tif")
            tifffile.imwrite(masks_path, masks.astype(np.uint16))

            torch.cuda.empty_cache()
        except Exception as e:
            print(f"⚠️ Error processing {fname}: {e}")
        pbar.update(1)

print("✅ Batch analysis complete.")

Found 93 TIFF stacks for analysis.


Processing TIFF stacks: 100%|███████████████████████████████████████████████████████████████████████| 93/93 [18:15<00:00, 11.78s/it]

✅ Batch analysis complete.
